In [1]:
# === SEKCJA 1: IMPORT I KONFIGURACJA ===
import pandas as pd
import numpy as np
import mlflow
from sklearn.feature_extraction.text import TfidfVectorizer
from pycaret.classification import setup, compare_models, tune_model, finalize_model, save_model, pull
from IPython.display import display

# Konfiguracja MLflow
MLFLOW_EXPERIMENT_NAME = 'Investoro_District_Classification_v3'
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
print(f"MLflow ustawiony. Eksperyment: '{MLFLOW_EXPERIMENT_NAME}'")

# Wczytanie danych
df_original = pd.read_csv('data.csv', sep=',')
print(f"Wczytano dane. Kształt: {df_original.shape}")

MLflow ustawiony. Eksperyment: 'Investoro_District_Classification_v3'
Wczytano dane. Kształt: (20092, 51)


In [2]:
# === SEKCJA 2: CZYSZCZENIE I INŻYNIERIA CECH ===

# --- Krok 2a: Podstawowe czyszczenie ---
print(f"Oryginalny rozmiar danych: {df_original.shape}")
df_cleaned = df_original.copy()
df_cleaned.dropna(subset=['Area', 'Price', 'Location', 'Description'], inplace=True)

Q1_price = df_cleaned["Price"].quantile(0.25)
Q3_price = df_cleaned["Price"].quantile(0.75)
IQR_price = Q3_price - Q1_price
df_cleaned = df_cleaned[~((df_cleaned["Price"] < (Q1_price - 1.5 * IQR_price)) | (df_cleaned["Price"] > (Q3_price + 1.5 * IQR_price)))]

Q1_area = df_cleaned["Area"].quantile(0.25)
Q3_area = df_cleaned["Area"].quantile(0.75)
IQR_area = Q3_area - Q1_area
df_cleaned = df_cleaned[~((df_cleaned["Area"] < (Q1_area - 1.5 * IQR_area)) | (df_cleaned["Area"] > (Q3_area + 1.5 * IQR_area)))]
print(f"Rozmiar po czyszczeniu i usunięciu outlierów: {df_cleaned.shape}")

# --- Krok 2b: Ekstrakcja Dzielnicy (Tworzenie celu) ---
def extract_district(location_str):
    if not isinstance(location_str, str): return np.nan
    parts = [part.strip() for part in location_str.split(',')]
    if len(parts) >= 3: return f"{parts[1]}, {parts[2]}"
    elif len(parts) == 2: return parts[1]
    else: return parts[0]

df_cleaned['District'] = df_cleaned['Location'].apply(extract_district)
df_cleaned.dropna(subset=['District'], inplace=True)
print(f"Dodano kolumnę 'District'. Liczba unikalnych wartości: {df_cleaned['District'].nunique()}")

# --- Krok 2c: Tworzenie cech TF-IDF ---
vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 2), min_df=5, max_df=0.95)
df_cleaned.reset_index(drop=True, inplace=True) # Reset indeksu PRZED transformacją
tfidf_features = vectorizer.fit_transform(df_cleaned['Description'])
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=['des_tfidf_' + name for name in feature_names])

# Łączymy w jeden finalny DataFrame
df_final = pd.concat([df_cleaned, tfidf_df], axis=1)
print(f"Dodano cechy TF-IDF. Finalny kształt zbioru: {df_final.shape}")

Oryginalny rozmiar danych: (20092, 51)
Rozmiar po czyszczeniu i usunięciu outlierów: (18612, 51)
Dodano kolumnę 'District'. Liczba unikalnych wartości: 444
Dodano cechy TF-IDF. Finalny kształt zbioru: (18612, 152)


In [3]:
# === SEKCJA 3: OSTATECZNE PRZYGOTOWANIE I SETUP (FINALNA WERSJA) ===

# --- Krok 3a: Filtrowanie rzadkich DZIELNIC ---
MIN_SAMPLES_PER_DISTRICT = 5
district_counts = df_final['District'].value_counts()
districts_to_remove = district_counts[district_counts < MIN_SAMPLES_PER_DISTRICT].index
df_model_ready = df_final[~df_final['District'].isin(districts_to_remove)].copy()

print(f"Użyto progu MIN_SAMPLES_PER_DISTRICT = {MIN_SAMPLES_PER_DISTRICT}")
print(f"Usunięto {len(districts_to_remove)} rzadkich dzielnic. Pozostało {len(df_model_ready)} wierszy.")
print(f"Liczba klas (dzielnic) do predykcji: {df_model_ready['District'].nunique()}")

# --- Krok 3b: Podział na zbiór treningowy i testowy ---
data_train = df_model_ready.sample(frac=0.9, random_state=1122)
data_test = df_model_ready.drop(data_train.index)
print(f"\nPodział danych -> Treningowe: {data_train.shape}, Testowe: {data_test.shape}")

# --- Krok 3c: Definicja list cech ---
numeric_features = [c for c in data_train.columns if c.startswith('des_tfidf_')] + ['Area', 'NumberOfRooms', 'Floor', 'Floors']
categorical_features = ['BuildingType', 'BuildingCondition', 'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom', 'VoivodeshipNumber', 'CountyNumber']
date_features = ['BuiltYear']

numeric_features = [f for f in numeric_features if f in data_train.columns]
categorical_features = [f for f in categorical_features if f in data_train.columns]
date_features = [f for f in date_features if f in data_train.columns]

# --- Krok 3d: Jawna konwersja typów przed setup ---
if 'BuiltYear' in date_features:
    data_train['BuiltYear'] = pd.to_datetime(data_train['BuiltYear'], errors='coerce')
    data_test['BuiltYear'] = pd.to_datetime(data_test['BuiltYear'], errors='coerce')

for col in categorical_features:
    data_train[col] = data_train[col].astype(str).fillna('missing')
    data_test[col] = data_test[col].astype(str).fillna('missing')
print("\nJawnie przekonwertowano typy danych.")

# --- Krok 3e: Setup PyCaret ---
print("\nRozpoczynam setup PyCaret...")
ignore_features_list = [
    'SaleId', 'OriginalId', 'Location', 'Description', 'Title', 'Link', 'Price', 
    'NewestDate', 'MainImage', 'OtherImages', 'EncryptedId'
]
ignore_features_list.extend([c for c in data_train.columns if c.startswith('Date') and c != 'BuiltYear'])
ignore_features_final = [col for col in ignore_features_list if col in data_train.columns]

s = setup(
    data=data_train,
    test_data=data_test,
    target='District',
    session_id=1122,
    log_experiment=False,
    
    n_jobs=4, # <-- NAJWAŻNIEJSZA ZMIANA JEST TUTAJ
    
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    date_features=date_features,
    ignore_features=ignore_features_final,
    
    html=False,
    verbose=True
)

Użyto progu MIN_SAMPLES_PER_DISTRICT = 5
Usunięto 205 rzadkich dzielnic. Pozostało 18177 wierszy.
Liczba klas (dzielnic) do predykcji: 239

Podział danych -> Treningowe: (16359, 152), Testowe: (1818, 152)

Jawnie przekonwertowano typy danych.

Rozpoczynam setup PyCaret...
                    Description  \
0                    Session id   
1                        Target   
2                   Target type   
3                Target mapping   
4           Original data shape   
5        Transformed data shape   
6   Transformed train set shape   
7    Transformed test set shape   
8               Ignore features   
9              Numeric features   
10                Date features   
11         Categorical features   
12     Rows with missing values   
13                   Preprocess   
14              Imputation type   
15           Numeric imputation   
16       Categorical imputation   
17     Maximum one-hot encoding   
18              Encoding method   
19               Fold Gener

In [4]:
# === SEKCJA 4: PORÓWNANIE WYBRANYCH MODELI (GŁÓWNY TRENING) ===
import mlflow
from pycaret.classification import compare_models, pull

# Upewnij się, że nie ma aktywnego runu
if mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run(run_name="District_Classification_Compare_Run") as run:
    print(f"Rozpoczęto run w MLflow: {run.info.run_id}")
    print("Rozpoczynam porównywanie wybranych, silnych modeli...")
    
    # Lista solidnych i popularnych modeli.
    # Usunęliśmy te, które mogą być niestabilne przy dużej liczbie klas.
    models_to_compare = ['lightgbm', 'xgboost', 'rf', 'et', 'gbc', 'lr']
    
    best_model = compare_models(
        include=models_to_compare, 
        sort='F1' # Sortujemy po F1-score, co jest dobrą metryką dla niezbalansowanych klas
    )
    
    print("\nPorównanie modeli zakończone.")
    
    # Wyciągnij i wyświetl tabelę z wynikami
    all_models_metrics = pull()
    display(all_models_metrics)
    
    if not all_models_metrics.empty:
        all_models_metrics.to_csv("compare_models_results.csv")
        mlflow.log_artifact("compare_models_results.csv")
        print(f"\nNajlepszy zidentyfikowany model: {best_model}")
    else:
        print("\nUWAGA: Żaden z testowanych modeli nie zakończył treningu pomyślnie.")

2025/06/15 00:04:01 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Rozpoczęto run w MLflow: eb33d9d9ba8944088f12efaedecafe09
Rozpoczynam porównywanie wybranych, silnych modeli...


                                    Model  Accuracy  AUC  Recall   Prec.  \
xgboost         Extreme Gradient Boosting    0.9796  0.0  0.9796  0.9737   
rf               Random Forest Classifier    0.9071  0.0  0.9071  0.8764   
lightgbm  Light Gradient Boosting Machine    0.1865  0.0  0.1865  0.1638   

              F1   Kappa     MCC  TT (Sec)  
xgboost   0.9758  0.9783  0.9784    14.550  
rf        0.8858  0.9007  0.9010     2.307  
lightgbm  0.1200  0.0967  0.1074    10.387  

Porównanie modeli zakończone.


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9796,0.0,0.9796,0.9737,0.9758,0.9783,0.9784,14.550
rf,Random Forest Classifier,0.9071,0.0,0.9071,0.8764,0.8858,0.9007,0.9010,2.307
lightgbm,Light Gradient Boosting Machine,0.1865,0.0,0.1865,0.1638,0.1200,0.0967,0.1074,10.387



Najlepszy zidentyfikowany model: XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=4, num_parallel_tree=None, ...)


In [5]:
# === SEKCJA 5: TUNING (WERSJA PROSTA) ===
from pycaret.classification import tune_model, pull

print("\nRozpoczynam tuning...")

# Ta funkcja automatycznie odziedziczy ustawienie n_jobs=4 z setup()
tuned_model = tune_model(best_model, n_iter=10, optimize='F1')

print("\nTuning zakończony.")
display(pull())


Rozpoczynam tuning...


Processing:   0%|                                                                                | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
      Accuracy  AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                       
0       0.9688  0.0  0.9688  0.9640  0.9643  0.9668  0.9668
1       0.9670  0.0  0.9670  0.9538  0.9592  0.9648  0.9649
2       0.9743  0.0  0.9743  0.9629  0.9676  0.9726  0.9727
3       0.9694  0.0  0.9694  0.9629  0.9648  0.9675  0.9675
4       0.9682  0.0  0.9682  0.9568  0.9616  0.9662  0.9662
5       0.9670  0.0  0.9670  0.9553  0.9596  0.9649  0.9649
6       0.9707  0.0  0.9707  0.9624  0.9651  0.9688  0.9688
7       0.9658  0.0  0.9658  0.9563  0.9594  0.9636  0.9636
8       0.9707  0.0  0.9707  0.9573  0.9627  0.9688  0.9688
9       0.9706  0.0  0.9706  0.9579  0.9634  0.9687  0.9688
Mean    0.9693  0.0  0.9693  0.9590  0.9628  0.9673  0.9673
Std     0.0024  0.0  0.0024  0.0035  0.0027  0.0025  0.0025

Tuning zak

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9688,0.0,0.9688,0.9640,0.9643,0.9668,0.9668
1,0.9670,0.0,0.9670,0.9538,0.9592,0.9648,0.9649
2,0.9743,0.0,0.9743,0.9629,0.9676,0.9726,0.9727
3,0.9694,0.0,0.9694,0.9629,0.9648,0.9675,0.9675
4,0.9682,0.0,0.9682,0.9568,0.9616,0.9662,0.9662
5,0.9670,0.0,0.9670,0.9553,0.9596,0.9649,0.9649
6,0.9707,0.0,0.9707,0.9624,0.9651,0.9688,0.9688
7,0.9658,0.0,0.9658,0.9563,0.9594,0.9636,0.9636
8,0.9707,0.0,0.9707,0.9573,0.9627,0.9688,0.9688


In [6]:
# === SEKCJA 6: FINALIZACJA I ZAPIS ===
from pycaret.classification import finalize_model, save_model

if tuned_model is not None:
    print("\nFinalizowanie modelu...")
    final_model = finalize_model(tuned_model)
    print(f"Model sfinalizowany: {final_model}")

    save_model(final_model, 'district_classifier_final_v4') # Nowa wersja
    print("\nFinalny model został zapisany do pliku.")
else:
    print("\nTuning nie powiódł się, model nie został sfinalizowany ani zapisany.")


Finalizowanie modelu...
Model sfinalizowany: Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('date_feature_extractor',
                 TransformerWrapper(exclude=None, include=['BuiltYear'],
                                    transformer=ExtractDateTimeFeatures(features=['day',
                                                                                  'month',
                                                                                  'year']))),
                ('numerical_imputer',
                 TransformerWrapper(exclude=Non...
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_t

In [7]:
# === SEKCJA 8: ZASTOSOWANIE MODELU I PREDYKCJA (POPRAWIONA) ===
from pycaret.classification import load_model, predict_model

print("Krok 1: Wczytywanie zapisanego, finalnego modelu...")
saved_final_model = load_model('district_classifier_final_v3')
print("Model wczytany pomyślnie.")

# --- KLUCZOWA POPRAWKA: Jawna konwersja typu PRZED predykcją ---
# Upewniamy się, że dane wejściowe mają dokładnie taki format, jakiego oczekuje pipeline
data_for_prediction = df_model_ready.copy()
data_for_prediction['BuiltYear'] = pd.to_datetime(data_for_prediction['BuiltYear'], errors='coerce')
print("\nTyp kolumny 'BuiltYear' został poprawnie ustawiony na datetime.")
# --- KONIEC POPRAWKI ---

print("\nKrok 2: Generowanie predykcji na całym przygotowanym zbiorze danych...")
# Przekazujemy do funkcji dane z poprawnym typem
predictions = predict_model(saved_final_model, data=data_for_prediction)
print("Predykcje wygenerowane.")

# Krok 3: Zmiana nazwy kolumny z predykcjami
predictions.rename(columns={'prediction_label': 'predykcja_lokalizacji'}, inplace=True)
print("Zmieniono nazwę kolumny na 'predykcja_lokalizacji'.")

# Zapisujemy wynik do nowej zmiennej
df_with_predictions = predictions.copy()

Krok 1: Wczytywanie zapisanego, finalnego modelu...
Transformation Pipeline and Model Successfully Loaded
Model wczytany pomyślnie.

Typ kolumny 'BuiltYear' został poprawnie ustawiony na datetime.

Krok 2: Generowanie predykcji na całym przygotowanym zbiorze danych...
                       Model  Accuracy     AUC  Recall   Prec.      F1  \
0  Extreme Gradient Boosting    0.3504  0.8156  0.3504  0.3344  0.3069   

    Kappa     MCC  
0  0.3006  0.3066  
Predykcje wygenerowane.
Zmieniono nazwę kolumny na 'predykcja_lokalizacji'.


In [8]:
# === SEKCJA 9: WERYFIKACJA I ZAPIS WYNIKÓW ===

print("Krok 4: Wyświetlanie przykładowych 25 wierszy z predykcjami...")
# Wybieramy tylko kilka najważniejszych kolumn do wyświetlenia, aby było czytelniej
display_columns = [
    'Price', 
    'Location', 
    'District',              # Oryginalna, prawdziwa dzielnica
    'predykcja_lokalizacji'  # Nasza nowa, przewidziana dzielnica
]
display(df_with_predictions[display_columns].head(25))

# Krok 5: Zapis całego DataFrame z predykcjami do pliku CSV
output_filename = 'data_loc.csv'
print(f"\nKrok 5: Zapisywanie danych z predykcjami do pliku '{output_filename}'...")
df_with_predictions.to_csv(output_filename, index=False, sep=',')
print("Zapisano pomyślnie!")

Krok 4: Wyświetlanie przykładowych 25 wierszy z predykcjami...


,Price,Location,District,predykcja_lokalizacji
0,409000.0,"Lubuskie, Żarski, Jasień","Żarski, Jasień","Żagański, Żagań"
1,664000.0,"Lubuskie, Żarski, Jasień","Żarski, Jasień","Żagański, Żagań"
2,190000.0,"Lubuskie, Żagański, Gozdnica","Żagański, Gozdnica",Gorzów Wielkopolski
3,339000.0,"Lubuskie, Gorzów Wielkopolski",Gorzów Wielkopolski,Gorzów Wielkopolski
4,299000.0,"Lubuskie, Gorzów Wielkopolski",Gorzów Wielkopolski,Zielona Góra
5,309000.0,"Lubuskie, Wschowski, Wschowa","Wschowski, Wschowa","Zielonogórski, Sulechów"
6,570000.0,"Lubuskie, Gorzów Wielkopolski",Gorzów Wielkopolski,Gorzów Wielkopolski
7,598000.0,"Lubuskie, Zielona Góra",Zielona Góra,Gorzów Wielkopolski
8,360000.0,"Lubuskie, Świebodziński, Świebodzin, Osiedle Ł...","Świebodziński, Świebodzin","Świebodziński, Świebodzin"
10,280000.0,"Lubuskie, Żagański, Żagań","Żagański, Żagań","Żagański, Żagań"



Krok 5: Zapisywanie danych z predykcjami do pliku 'data_loc.csv'...
Zapisano pomyślnie!
